In [1]:
import os
import sys
import conda
import importlib

### Helper Functions

In [2]:
def check_io(io):
    return bool(io.stdout) and not bool(io.stderr)

def check_import(name):
    try:
        m = importlib.import_module(name)
        res = m.__version__
    except ImportError as e:
        res = repr(e)
    return res

## Check conda path

The installation path of anaconda cannot contain any space characters (```' '```).

In [3]:
conda_path_ok = conda.CONDA_PACKAGE_ROOT.find(' ') < 0

In [4]:
matplotlib = check_import('matplotlib')
numpy = check_import('numpy')
pandas = check_import('pandas')
pip = check_import('pip')

In [5]:
%%capture matplotlib_backend
%matplotlib

In [6]:
%%capture git
!git --version

In [7]:
%%capture vscode
!code --version

In [8]:
%%capture python
!python --version

In [9]:
%%capture conda
!conda --version

In [10]:
%%capture jupyter
!jupyter --version

## Print Report